## Единый агрегатор данных о продажах

**Описание проекта:**

Разработка скрипта (функции process_files) для автоматической обработки и объединения данных о продажах (чеков) из различных магазинов в единый набор данных. Сохранение объединённого чека (отсортированного по дате, а затем по продукту) под названием combined_data.csv.

**Требования:**

**Отбор файлов:** Обрабатывать только файлы, соответствующие шаблону имени: ГГГГ-ММ-ДД-ЧЧ-ММ-idклиента.csv, где:

- **ГГГГ** — год,

- **ММ** — месяц,

- **ДД** — день,

- **ЧЧ** — час,

- **ММ** — минута,

- **idклиента** — уникальный идентификатор клиента.

**Обработка данных:** Отфильтровать ненужную информацию и привести данные к единому формату.

**Объединение данных:** Создать единый файл combined_data.csv, содержащий все чеки из обработанных файлов. Первая строка первого файла должна стать строкой заголовка.

**Сохранение результата:** Сохранить итоговый файл в указанную папку для дальнейшего анализа.

**Цель:** Получить унифицированный, очищенный и структурированный набор данных для анализа продаж.

## Импорт библиотек

In [ ]:
import zipfile
import pandas as pd
import re
import glob
import os
import csv

## Загрузка папки с файлами.

В папке `reports_main` будут храниться все присланные магазинами чеки.

In [ ]:
!wget https://github.com/vs8th/reports/archive/main.zip

with zipfile.ZipFile("main.zip", 'r') as zip_ref:
    zip_ref.extractall("/content")

!rm main.zip

--2025-03-10 12:42:26--  https://github.com/vs8th/reports/archive/main.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/Vs8th/reports/zip/refs/heads/main [following]
--2025-03-10 12:42:26--  https://codeload.github.com/Vs8th/reports/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.112.9
Connecting to codeload.github.com (codeload.github.com)|140.82.112.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [ <=>                ]   3.22K  --.-KB/s    in 0s      

2025-03-10 12:42:26 (45.9 MB/s) - ‘main.zip’ saved [3296]



Образец подходящего для объединения чека

In [ ]:
df = pd.read_csv('reports-main/2023-02-17-05-38-2.csv', sep=";")
df

,date,product,store,cost
0,2023-02-17,product_0,store_2,10
1,2023-02-17,product_1,store_2,20
2,2023-02-17,product_2,store_2,30
3,2023-02-17,product_3,store_2,40
4,2023-02-17,product_4,store_2,50


## Создание функции


In [ ]:
def process_files(src_folder, dest_folder):

    # Создание папки dest_folder, если она не существует
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    # Поиск файлов с расширением .csv
    csv_files = glob.glob(os.path.join(src_folder, '*.csv'))

    # Фильтр по шаблону названия файла
    pattern = re.compile(r'\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d+\.csv')
    filtered_files = [file for file in csv_files if pattern.search(file)]

    # Заданный заголовок
    required_columns = ['date', 'product', 'store', 'cost']

    # Открытие итогового файла для записи
    output_path = os.path.join(dest_folder, 'combined_data.csv')
    with open(output_path, 'w') as output_file:
        writer = csv.writer(output_file, delimiter=',')
        writer.writerow(required_columns)

        # Обработка каждого файла
        for file in filtered_files:
            with open(file, 'r') as current_file:
                reader = csv.DictReader(current_file, delimiter=';')

                # Проверка наличия нужного заголовка в файле
                if all(col in reader.fieldnames for col in required_columns):
                    # Записывание данных из файла
                    for row in reader:
                        # Выбор только нужных столбцов
                        filtered_row = [row[col] for col in required_columns]
                        writer.writerow(filtered_row)

    # Сортировка итогового файла по всем столбцам
    df = pd.read_csv(output_path)
    df.sort_values(by=['date', 'product', 'store'], inplace=True)
    df.to_csv(output_path, index=False)

## Проверка


In [ ]:
# Выполнение функции

src_folder = 'reports-main'
dest_folder = 'comb_reports'
process_files(src_folder, dest_folder)

In [ ]:
# Скачивание файла с эталонным ответом

!wget https://gist.github.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv

user_answer = pd.read_csv(f'{dest_folder}/combined_data.csv')
correct_answer = pd.read_csv('data.csv')

--2025-03-10 12:42:27--  https://gist.github.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv
Resolving gist.github.com (gist.github.com)... 140.82.114.4
Connecting to gist.github.com (gist.github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv [following]
--2025-03-10 12:42:27--  https://gist.githubusercontent.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 984 [text/plain]
Saving to: ‘data.csv.18’

data.csv.18         100%[===================>]     984  --.-KB/s    in 0s      

2025-03-10 12:42:27 (32.3 MB/s) - ‘data.csv.18’ saved [984/984]



In [ ]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
  assert user_answer.columns.equals(correct_answer.columns), 'Названия столбцов не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!')

Поздравляем, Вы справились и успешно прошли все проверки!
